之所以起这个名字，实际上股票买卖问题的本质即，对于数组中的所有元素，想找到两个元素，满足
- num[i] - num[j] 这一 metric 最大，即 max(num[i] - num[j])；
- i 必须在 j 之后，即 i > j;

这是一个标准的最优化目标，一个目标函数，一个 constraint；不论这两个事情怎么换（比如，改成 max(num[i] * num[j])，或者改成 i > j），其思路是不变的。

121. 买卖股票的最佳时机

https://leetcode.cn/problems/best-time-to-buy-and-sell-stock/description/?envType=study-plan-v2&envId=top-interview-150

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n, best = len(prices), 0
        if n < 2:
            return best
        for i in range(n):
            for j in range(i + 1, n):
                diff = prices[j] - prices[i]
                best = max(diff, best)
        return best

如上自然是最好想的写法，非常直觉明了，但是时间复杂度是二阶的。那么更好的做法是什么？这里要反思的是，哪些操作是冗余的，很明显，我们对于元素的访问是冗余的，也就是说访问的次数**本可以更少的**。

我们现在要做的变换是，将以 max(num[i] - num[j]) 和 i > j 为目标的 (i, j) 遍历方法，改为只遍历 i，并基于新的变量记录 min(num[:i])，在数学上是一种将约束条件放回到目标函数的做法。实际上，就是维护“到目前为止的最低买入价”，并实时更新最大利润。如下

$$
\max \left( num[i] - num[j] \right) \quad \text{s.t.} \quad i > j \\
= \max (num[i] - \min(num[:i])) \quad \forall\, i > 0
$$

In [ ]:
from typing import List

class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        min_price = float("inf")
        max_profit = 0

        for p in prices:
            if p < min_price:
                min_price = p
            else:
                max_profit = max(max_profit, p - min_price)

        return max_profit

或者如下，在实现上只有一次更新的相位差。但要学会正无穷的写法，即 float("inf")。

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        best, lowest = 0, 100000
        if len(prices) < 2:
            return best
        for price in prices:
            best = max(price - lowest, best)
            lowest = min(lowest, price)
        return best

122. 买卖股票的最佳时机 II

https://leetcode.cn/problems/best-time-to-buy-and-sell-stock-ii/description/?envType=study-plan-v2&envId=top-interview-150

这题有点所谓仿真思想了，也就是自己炒过股，知道低买高卖，然后自然想要去设置指针跟踪低点高点，不过高点当时就操作了，完全没必要跟踪，因此就一个指针 lowest 即可；此外还需要一个状态跟踪，就是当前是涨是跌的 is_up，剩下的就是标准的状态机思想。凡是涉及到拐点、升降变化的，大多要用状态机来捕捉拐点。

这道题一遍过，表扬自己注意了边界条件；但是也是因为题例给的好，如果没给的话也应该想到全升的情况。这道题和第一题的目标其实完全不同，它也是想要最大化利差，但不是一个全局 global 目标，而是可以多局部 local 求和的目标。如果还可以做空的话，是不是可以变成双向的遍历？

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n = len(prices)
        if n < 2:
            return 0
        total = 0
        is_up, lowest = False, 0
        for i in range(1, n):
            if prices[i] >= prices[i - 1]: # rising
                if not is_up:
                    is_up = True
                    lowest = i - 1
                if is_up and i == (n - 1):
                    total += prices[i] - prices[lowest] 
            else:
                if is_up:
                    is_up = False
                    total += prices[i - 1] - prices[lowest]      
        return total